In [3]:
using Clang

In [12]:
sdk_dir = `xcrun --show-sdk-path` |> open |> readchomp |> String

"/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk"

In [16]:
trans_unit = Clang.parse_header(Index(), "050_test.h", ["-x","objective-c","-fmodules", "-isysroot", sdk_dir])

TranslationUnit(Ptr{Clang.LibClang.CXTranslationUnitImpl} @0x000000013de2eff0, Index(Ptr{Nothing} @0x000000013de57040, 0, 1))

In [17]:
root_cursor = Clang.getTranslationUnitCursor(trans_unit)

CLCursor (CLTranslationUnit) 050_test.h

In [23]:
printind(ind::Int, st...) = println(join([repeat(" ", 2*ind), st...]))

printobj(cursor::CLCursor) = printobj(0, cursor)
printobj(t::CLType) = "$(typeof(t)) $(spelling(t))"
printobj(t::CLInt) = t
printobj(t::CLPointer) = Clang.getPointeeType(t)
printobj(ind::Int, t::CLType) = printind(ind, printobj(t))

function printobj(ind::Int, cursor::Union{CLFieldDecl, CLParmDecl})
    printind(ind+1, typeof(cursor), " ", printobj(Clang.getCursorType(cursor)), " ", name(cursor))
end

function printobj(ind::Int, node::Union{CLCursor, CLStructDecl, CLCompoundStmt,
                                        CLFunctionDecl, CLBinaryOperator})
    printind(ind, " ", typeof(node), " ", name(node))
    for c in children(node)
        printobj(ind + 1, c)
    end
end

printobj (generic function with 7 methods)

In [24]:
printobj(root_cursor)

 CLTranslationUnit 050_test.h
   CLObjCClassRef Protocol
   CLStructDecl 
      CLFieldDecl CLType (CLInt)  a
      CLFieldDecl CLType (CLInt)  b
   CLTypedefDecl Bar
     CLStructDecl 
        CLFieldDecl CLType (CLInt)  a
        CLFieldDecl CLType (CLInt)  b
   CLFunctionDecl m1(Bar)
      CLParmDecl CLTypedef Bar in
     CLCompoundStmt 


       CLReturnStmt 
         CLBinaryOperator 
           CLUnexposedExpr a
             CLMemberRefExpr a
               CLDeclRefExpr in
           CLUnexposedExpr b
             CLMemberRefExpr b
               CLDeclRefExpr in
   CLFunctionDecl m2(Bar)
     CLTypeRef Bar
      CLParmDecl CLTypedef Bar in
     CLCompoundStmt 
       CLCompoundAssignOperator 
         CLMemberRefExpr a
           CLDeclRefExpr in
         CLIntegerLiteral 
       CLCompoundAssignOperator 
         CLMemberRefExpr b
           CLDeclRefExpr in
         CLIntegerLiteral 
       CLReturnStmt 
         CLUnexposedExpr in
           CLDeclRefExpr in


   CLObjCInterfaceDecl Foo
     CLObjCInstanceMethodDecl m0:
        CLParmDecl CLType (CLInt)  x
     CLObjCInstanceMethodDecl m1:
        CLParmDecl CLTypedef Bar x
     CLObjCInstanceMethodDecl m2:
       CLTypeRef Bar
        CLParmDecl CLTypedef Bar x
   CLObjCImplementationDecl Foo
     CLObjCInstanceMethodDecl m0:
        CLParmDecl CLType (CLInt)  x
       CLCompoundStmt 
         CLReturnStmt 
           CLBinaryOperator 
             CLIntegerLiteral 
             CLUnexposedExpr x
               CLDeclRefExpr x
     CLObjCInstanceMethodDecl m1:
        CLParmDecl CLTypedef Bar x
       CLCompoundStmt 
         CLReturnStmt 
           CLBinaryOperator 
             CLUnexposedExpr a
               CLMemberRefExpr a
                 CLDeclRefExpr x
             CLUnexposedExpr b
               CLMemberRefExpr b
                 CLDeclRefExpr x
     CLObjCInstanceMethodDecl m2:
       CLTypeRef Bar
        CLParmDecl CLTypedef Bar x
       CLCompoundStmt 
         CLCompoundAs